In [34]:
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
import os
import json
import random

In [35]:
file_path = 'demography.json'
demography = None
with open(file_path, 'r') as file:
    demography = json.load(file)
demography

{'age': [{'category': 'Teens', 'start': 13, 'end': 19},
  {'category': 'Young Adults', 'start': 20, 'end': 35},
  {'category': 'Adults', 'start': 35, 'end': 50},
  {'category': 'Seniors', 'start': 51, 'end': 70}],
 'genders': ['Female', 'Male', 'Non-binary'],
 'ethnicities': ['Hispanic/Latino',
  'Caucasian',
  'Asian',
  'African American',
  'Mixed race',
  'Native American'],
 'locations': ['North America',
  'Europe',
  'Asia',
  'South America',
  'Africa',
  'Oceania'],
 'professionalBackgrounds': ['Blogger',
  'Professional Critic',
  'Film Student',
  'Casual Moviegoer',
  'YouTuber',
  'Industry Professional'],
 'educations': ['High School',
  "Bachelor's Degree",
  "Master's Degree",
  'Doctorate',
  'Self-taught'],
 'incomeLevels': ['Low Income', 'Middle Income', 'High Income']}

In [42]:
def get_random_persona(demography, seed=None):
    if seed:
        random.seed(seed)
    while True:
        permutation = {}
        permutation['age'] = random.choice(demography['age'])
        permutation['gender'] = random.choice(demography['genders'])
        permutation['ethnicity'] = random.choice(demography['ethnicities'])
        permutation['location'] = random.choice(demography['locations'])
        permutation['professionalBackground'] = random.choice(demography['professionalBackgrounds'])
        permutation['education'] = random.choice(demography['educations'])
        permutation['incomeLevel'] = random.choice(demography['incomeLevels'])
        yield permutation

In [52]:
random_persona_generator = get_random_persona(demography, 42)
next(random_persona_generator)

{'age': {'category': 'Teens', 'start': 13, 'end': 19},
 'gender': 'Female',
 'ethnicity': 'Native American',
 'location': 'Asia',
 'professionalBackground': 'Professional Critic',
 'education': "Bachelor's Degree",
 'incomeLevel': 'Low Income'}

In [63]:
def generate_demography_prompt_input(persona):
    return {
      "ageStart": persona["age"]["start"],
      "ageEnd": persona["age"]["end"],
      "gender": persona["gender"],
      "ethnicity": persona["ethnicity"],
      "location": persona["location"],
      "profession": persona["professionalBackground"],
      "education": persona["education"],
      "income": persona["incomeLevel"]
    }

In [71]:
individual_prompt = PromptTemplate.from_template("""
You're an individual with a specific age, gender, ethnic background coming from a defined geographic location.

Those parameters are specified within the following block.
It starts by BEGIN and ends by END.

BEGIN demography
age range start: {ageStart}
age range end: {ageEnd}
gender: {gender}
ethnicity: {ethnicity}
location: {location}
professional background: {profession}
education: {education}
income: {income}
END demography
""")

In [72]:
review_prompt = PromptTemplate.from_template("""
Write a review about Witcher season 2? 
""")

In [73]:
full_prompt = individual_prompt + review_prompt

In [65]:
model=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106", openai_api_key=os.getenv("OPENAI_API_KEY"))

In [74]:
full_prompt.invoke(generate_demography_prompt_input(next(random_persona_generator)))

StringPromptValue(text="\nYou're an individual with a specific age, gender, ethnic background coming from a defined geographic location.\n\nThose parameters are specified within the following block.\nIt starts by BEGIN and ends by END.\n\nBEGIN demography\nage range start: 20\nage range end: 35\ngender: Male\nethnicity: Mixed race\nlocation: Asia\nprofessional background: Blogger\neducation: Bachelor's Degree\nincome: High Income\nEND demography\n\nWrite a review about Witcher season 2? \n")

In [69]:
#question_prompt = PromptTemplate.from_template("Tell me a bit about your background?")

In [77]:
chain = full_prompt | model 

In [78]:
chain.invoke(demography_example_1)

AIMessage(content='As a 28-year-old Hispanic/Latina female blogger from North America, I recently had the opportunity to watch Witcher season 2 and I must say, it did not disappoint. The second season continued to captivate me with its intricate storytelling, stunning visuals, and compelling characters.\n\nThe character development of Geralt, Yennefer, and Ciri was truly impressive, and I found myself becoming even more invested in their individual journeys. The action sequences were intense and well-choreographed, keeping me on the edge of my seat throughout each episode.\n\nI also appreciated the expansion of the Witcher universe, as we were introduced to new magical creatures and realms that added depth to the overall storyline. The political intrigue and power struggles within the various kingdoms added an extra layer of complexity to the plot, keeping me engaged and eager to see how everything would unfold.\n\nOverall, Witcher season 2 exceeded my expectations and left me eagerly 